# Pipeline Implementation

This code will contain the implementation for our pipeline that combines both our object detection and our traffic prediction network

In [1]:
from object_detection.callable_implementation import write_csv
from object_detection.callable_implementation import process_image
from object_detection.callable_implementation import get_dt_info
from object_detection.callable_implementation import load_model
from object_detection.callable_implementation import load_tensor_img
from object_detection.render_video import images_to_video_ffmpeg

import requests
import cv2
import numpy as np
import imutils
import time
import os
import threading
from queue import Queue
import shutil

In [2]:
# load the model
model, vehicle_classes, device = load_model()

In [3]:
# read ip from text file
with open('my_ip.txt', 'r') as f:
    ip = f.read().strip()
print(ip)

192.168.4.111:8080


In [ ]:
# Replace the below URL with your own. Make sure to add "/shot.jpg" at the last.
url = f"http://{ip}/shot.jpg"

# Initialize variables
fps = 30  # Desired FPS for the output video
out_dir = "test_images"

# Create the output directory if it doesn't exist
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Queue for storing images that need to be annotated
image_queue = Queue()

# Function to process the image
def annotate(image_path):
    print(f"Annotating image: {image_path}")
    output_path = 'live_stream'
    image_base_name = os.path.basename(image_path).split('.')[0]
    outfile_name = f'{image_base_name}.jpg'
    img, img_tensor = load_tensor_img(image_path, device)
    data = process_image(model, vehicle_classes, img, img_tensor, output_path, outfile_name)
    time, day = get_dt_info()
    data["Time"] = time
    data["Day of the week"] = day
    write_csv(data, "test.csv")

# Function to process images from the queue
def process_annotations():
    while True:
        # Get an image path from the queue
        img_path = image_queue.get()
        if img_path is None:  # Sentinel value to stop the thread
            break
        # Call the annotation function for each image
        annotate(img_path)
        image_queue.task_done()

# Start the annotation processing thread
annotation_thread = threading.Thread(target=process_annotations, daemon=True)
annotation_thread.start()

# Timing control to achieve 30 FPS
prev_time = time.time()

# While loop to continuously fetch data from the URL
count = 0
while True:
    # Calculate the time difference
    current_time = time.time()
    time_diff = current_time - prev_time

    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    img = cv2.imdecode(img_arr, -1)
    img = imutils.resize(img, width=1000, height=1800)

    # Save the image with a timestamp to ensure uniqueness
    filename = f"{out_dir}/image_{count}.jpg"
    cv2.imwrite(filename, img)

    # Add the image path to the queue for annotation
    image_queue.put(filename)

    # Update the previous time to current time
    prev_time = current_time

    # Display the image
    cv2.imshow("Android_cam", img)

    # Press Esc key to exit
    if cv2.waitKey(1) == 27:
        break

    # Sleep to avoid high CPU usage (sleeping 1ms in case timing isn't perfect)
    time.sleep(1 / fps)
    count += 1

cv2.destroyAllWindows()

# Stop the annotation thread gracefully
image_queue.put(None)  # Sentinel to stop the thread
annotation_thread.join()


In [ ]:

date_time_info = get_dt_info()
images_to_video_ffmpeg("live_stream", f"live_stream_{date_time_info}.mp4", fps // 4)

# # remove live_stream folder and its contents
# shutil.rmtree("live_stream")

2024/11/27 12:58:45.063531 cmd_run.go:1138: WARNING: cannot start document portal: dial unix /run/user/1000/bus: connect: no such file or directory
ffmpeg version n4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix= --prefix=/usr --disable-debug --disable-doc --disable-static --enable-cuda --enable-cuda-sdk --enable-cuvid --enable-libdrm --enable-ffplay --enable-gnutls --enable-gpl --enable-libass --enable-libfdk-aac --enable-libfontconfig --enable-libfreetype --enable-libmp3lame --enable-libnpp --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopus --enable-libpulse --enable-sdl2 --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libv4l2 --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libxcb --enable-libxvid --enable-nonfree --enable-nvenc --enable-omx --enable-openal --enable-opencl --enable-runtime-cpudetect --enable-shared --enable-vaapi --enable-

Video saved as live_stream_('125844', 'Wednesday').mp4


frame=  104 fps= 59 q=-1.0 Lsize=    3663kB time=00:00:14.42 bitrate=2079.9kbits/s speed=8.25x    
video:3661kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.048488%
[libx264 @ 0x5633108098c0] frame I:6     Avg QP:19.43  size: 52417
[libx264 @ 0x5633108098c0] frame P:60    Avg QP:21.49  size: 37613
[libx264 @ 0x5633108098c0] frame B:38    Avg QP:23.49  size: 30984
[libx264 @ 0x5633108098c0] consecutive B-frames: 47.1%  9.6%  8.7% 34.6%
[libx264 @ 0x5633108098c0] mb I  I16..4:  4.1% 70.2% 25.7%
[libx264 @ 0x5633108098c0] mb P  I16..4:  6.2% 51.5% 13.7%  P16..4: 11.7%  9.4%  5.4%  0.0%  0.0%    skip: 2.1%
[libx264 @ 0x5633108098c0] mb B  I16..4:  1.6%  9.3%  4.7%  B16..8: 34.2% 19.9%  8.0%  direct:10.1%  skip:12.3%  L0:40.0% L1:40.7% BI:19.4%
[libx264 @ 0x5633108098c0] 8x8 transform intra:70.6% inter:56.9%
[libx264 @ 0x5633108098c0] coded y,uvDC,uvAC intra: 66.7% 60.3% 36.3% inter: 52.5% 43.0% 5.5%
[libx264 @ 0x5633108098c0] i16 v,h,dc,p: 32% 19%  6% 42%
